In [ ]:
!pip install pyarrowfs-adlgen2
!pip install azure-identity
!pip install azure-storage-file-datalake
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pwd

/content


In [ ]:
# import tensorflow_decision_forests as tfdf
import ydf
import numpy as np
import pandas as pd
import tensorflow as tf
import tf_keras
import os
import duckdb

import pyarrow as pa
import pyarrow.parquet as pq

from dotenv import load_dotenv
from pathlib import Path

In [ ]:
# Retrieve credentials from environment variables
# this is strictly used only in development
# load env variables
env_dir = Path('./').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [ ]:
MOTHERDUCK_TOKEN = os.environ.get("MOTHERDUCK_TOKEN")

In [ ]:
conn = duckdb.connect("md:", config={"motherduck_token": MOTHERDUCK_TOKEN})

In [ ]:
conn.sql("SELECT CURRENT_DATABASE()")

┌────────────────────┐
│ current_database() │
│      varchar       │
├────────────────────┤
│ my_db              │
└────────────────────┘

In [ ]:
conn.sql("USE signal_gender_predictor_db")

In [ ]:
conn.sql("SELECT CURRENT_DATABASE()")

┌────────────────────────────┐
│     current_database()     │
│          varchar           │
├────────────────────────────┤
│ signal_gender_predictor_db │
└────────────────────────────┘

convert the tables to numpy arrays

In [ ]:
# returns the pyarrow table as a dictionary e.g.
# {
#     'f1': [...],
#     'f2': [...],
#     ...
#     'fn': [...],
#     'label': [...]
# }
train_data_sc_sm_red_dict = conn.sql("SELECT * FROM train_data_sc_sm_red").fetchnumpy()
val_data_sc_sm_red_dict = conn.sql("SELECT * FROM val_data_sc_sm_red").fetchnumpy()
test_data_sc_sm_red_dict = conn.sql("SELECT * FROM test_data_sc_sm_red").fetchnumpy()

Cast the label of the numpy dictionary to an int instead of float

In [ ]:
train_data_sc_sm_red_dict["label"] = train_data_sc_sm_red_dict["label"].astype(int)
val_data_sc_sm_red_dict["label"] = val_data_sc_sm_red_dict["label"].astype(int)
test_data_sc_sm_red_dict["label"] = test_data_sc_sm_red_dict["label"].astype(int)

In [ ]:
ydf.verbose()
model = ydf.GradientBoostedTreesLearner(
    label="label",
    include_all_columns=True,
    task=ydf.Task.CLASSIFICATION,
).train(train_data_sc_sm_red_dict, val_data_sc_sm_red_dict)

max_vocab_count = -1 for column label, the dictionary will not be pruned by size.


Train model on 292 training examples and 54 validation examples


"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 292
Number of columns: 61

Number of columns by type:
	NUMERICAL: 60 (98.3607%)
	CATEGORICAL: 1 (1.63934%)

Columns:

NUMERICAL: 60 (98.3607%)
	1: "freq_kurt_imp" NUMERICAL mean:0.0950331 min:-2.73647 max:1.73015 sd:0.981947 dtype:DTYPE_FLOAT64
	2: "freq_skew_imp" NUMERICAL mean:-0.027617 min:-1.88086 max:1.75999 sd:1.03184 dtype:DTYPE_FLOAT64
	3: "freq_entropy_imp" NUMERICAL mean:0.045646 min:-3.12559 max:1.15568 sd:0.971616 dtype:DTYPE_FLOAT64
	4: "freq_mean_imp" NUMERICAL mean:-0.0724972 min:-6.80855 max:1.39331 sd:1.02984 dtype:DTYPE_FLOAT64
	5: "freq_median_imp" NUMERICAL mean:-0.0268643 min:-12.497 max:2.22414 sd:0.949349 dtype:DTYPE_FLOAT64
	6: "freq_min_imp" NUMERICAL mean:-0.0937558 min:-2.56508 max:0.797143 sd:1.07629 dtype:DTYPE_FLOAT64
	7: "freq

Model trained in 0:00:00.702945


In [ ]:
model.save("/tmp/signal-gender-predictor/")

# save yggdrasil model to temporary directory 

In [ ]:
signal_gender_predictor = ydf.load_model("/tmp/signal-gender-predictor/")
signal_gender_predictor

Loading model from /tmp/signal-gender-predictor/


In [ ]:
signal_gender_predictor.plot_tree()

In [ ]:
evaluation = signal_gender_predictor.evaluate(test_data_sc_sm_red_dict)

Engine "GradientBoostedTreesQuickScorerExtended" built


In [ ]:
evaluation

Label \ Pred,0,1
0,23,40
1,58,5
